In [2]:
# load
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import log_loss
from tqdm import tqdm
import lightgbm as lgb
from sklearn.model_selection import KFold
import re
tqdm.pandas(desc="my bar!")

pd.set_option('max_columns', 100)
pd.set_option('max_rows', 100)
def wlogloss(targets, preds):
    target_cols = ['any', 'epidural', 'subdural', 'subarachnoid', 'intraventricular', 'intraparenchymal']
    res = 0
    for col in target_cols:
        res += log_loss(targets[col], preds[col+'_pred'])
        if col == 'any':
            res += log_loss(targets[col], preds[col+'_pred'])
    res /= 7
    return res
target_cols = ['any', 'epidural', 'subdural', 'subarachnoid', 'intraventricular', 'intraparenchymal']


/usr/local/lib/python3.7/dist-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# train

In [3]:
import pickle
with open('test_10models_stackfeats.pickle', 'rb') as f:
    test_list = pickle.load(f)
with open('train_10model_stackfeats.pickle', 'rb') as f:
    train = pickle.load(f)


In [26]:
import re
X_cols = test_list[0].columns.drop(['ID'])
X_cols = [c for c in X_cols if not c.endswith('div')]
X_cols = [c for c in X_cols if len(re.findall('_7roll|_9roll|_3rollmin|_3rollmin|_3rollstd|_5rollstd|_3rollmax|_5rollmax', c)) == 0][:100]

In [27]:
len(X_cols)

100

In [28]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler = MinMaxScaler()
_mean = train[X_cols].mean()
a = train[X_cols].fillna(_mean)
scaler.fit(a)


MinMaxScaler(copy=True, feature_range=(0, 1))

In [29]:
%%time
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers.core import Dense, Activation, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint


model = Sequential()
model.add(Dense(1024, input_dim=len(X_cols)))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# we'll use categorical xent for the loss, and RMSprop as the optimizer
model.compile(loss='binary_crossentropy', optimizer='adam')

print("Training...")

from sklearn.linear_model import Ridge, LogisticRegression
stack_preds = []
res = []
pred_test = []
fi = {}
for c in target_cols:
    fi[c] = []
stack_preds_valid = []

for i in range(5):
    tr = train.query('folds != @i')
    tr[X_cols] = tr[X_cols].fillna(_mean)
    va = train.query('folds == @i')
    va[X_cols] = va[X_cols].fillna(_mean)
    print('fitting')
    preds = pd.DataFrame(np.zeros([len(va), 6]), columns = [c + '_pred' for c in target_cols])
    for tar_col in target_cols:
        es_cb = EarlyStopping(monitor='val_loss', patience=2, verbose=1, mode='auto')
        chkpt = f'stack2_{i}fold_{tar_col}.hdf5'
        cp_cb = ModelCheckpoint(filepath = chkpt, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
        model.fit(scaler.transform(tr[X_cols].values), tr[tar_col].values, nb_epoch=100, batch_size=512, validation_data=(scaler.transform(va[X_cols].values), va[tar_col].values), verbose=2, callbacks=[es_cb,cp_cb])
        model.load_weights(chkpt)
        preds[tar_col + '_pred'] = model.predict(scaler.transform(va[X_cols].values))
        pred_test.append(model.predict(scaler.transform(test_list[i][X_cols].values)))
        print(log_loss(va[tar_col], preds[tar_col + '_pred']))
        res.append(log_loss(va[tar_col], preds[tar_col + '_pred']))
        print('-'*80)
    print('='*80)
    stack_preds.append([va['ID'], preds])

Training...


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


fitting


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 531555 samples, validate on 133859 samples
Epoch 1/100
 - 5s - loss: nan - val_loss: nan

Epoch 00001: val_loss did not improve from inf
Epoch 2/100
 - 5s - loss: nan - val_loss: nan

Epoch 00002: val_loss did not improve from inf
Epoch 00002: early stopping


ValueError: Shapes (100, 1024) and (1080, 2048) are incompatible

In [20]:
def get_score(res):
    score = 0
    for i, r in enumerate(res):
        if i %6 == 0:
            score += 2*r
        else:
            score += r
    return score/5/7
get_score(res)

0.0

In [21]:
path = f'../prediction/appian/'
n_fold = 0
sub = pd.read_pickle(f'{path}/fold{n_fold}_ep2_test_tta5.pkl')
test = test_list[0]
preds_test_df = pd.DataFrame()
for n_fold in range(5):
    for n_target in range(6):
        preds_test_df[target_cols[n_target] + '_' + str(n_fold) + 'fold'] = pred_test[n_fold*6 + n_target].reshape(-1)
preds_test_df.index = test['ID'].values
preds_test_df = preds_test_df.loc[sub[0]['ids']]
pred_test_sorted_list = []
for i in range(30):
    pred_test_sorted_list.append(preds_test_df.iloc[:, i].values)
pred_test_df_list = []
for n_fold in range(5):
    tmp = np.zeros([len(test), 6])
    for n_tar in range(6):
        tmp[:, n_tar] = pred_test_sorted_list[6*n_fold + n_tar]
    pred_test_df = pd.DataFrame(tmp, columns = target_cols)
    pred_test_df_list.append(pred_test_df)
sub = pd.read_pickle(f'{path}/fold{n_fold}_ep2_test_tta5.pkl')
sub_original = pd.read_pickle(f'{path}/fold{n_fold}_ep2_test_tta5.pkl')
for i in range(5):
    sub[i]['outputs'] = pred_test_df_list[i][target_cols].values
for i in range(5):
    for ii in range(6):
        print(np.corrcoef(sub[i]['outputs'][:, ii], sub_original[i]['outputs'][:, ii])[0, 1])
path_to = '../stack_prediction/pred_test_1103_NN_add_appian_feats.pkl'
with open(path_to, 'wb') as f:
    pickle.dump(sub, f)
path_to = f'../stack_prediction/pred_valid_1103_NN_add_appian_feats.pkl'
with open(path_to, 'wb') as f:
    pickle.dump(stack_preds, f)

IndexError: list index out of range